# Solution Tutorial 4

### Setup

In [1]:
import numpy as np
from numpy.linalg import norm
np.set_printoptions(precision=4, floatmode='fixed', suppress=True, sign=' ', 
                    formatter={'int': lambda x: "{:> }".format(x)})

D = np.array([[1,0,2,+1],[1,2,0,-1],[1,2,2,+1],[1,1,1,-1]])
X = D[:,:-1]
Y = D[:, -1]
N, M = X.shape
beta0 = np.zeros((M)) #+ [-3,-1,2]
alpha0= np.zeros((N))
lam=1

# 1. Gradient descent

We vectorize the sum $\sum_\Omega y_nx_n$ with  $\Omega = \{n\in\{1,\ldots N\} \mid y_n (\beta_0 + \beta^T x_n)<1\}$ via masking

In [2]:
def grad(X, Y, beta):
    mask = Y*(X@beta)<1
    x_masked = X[mask]
    y_masked = Y[mask]
    Z = -x_masked.T @ y_masked
    return Z/len(X) + lam*beta, mask, Z

def GD(X, Y, beta0, lam=1, eta=0.5, maxiter=2):
    beta = beta0.copy()
    for t in range(maxiter):
        dbeta, mask, Z = grad(X, Y, beta)
        beta -= eta*dbeta 
        print(F"it: {t:2d}  mask: {mask.astype(int)}  Z:{Z}  B: {beta}  XB: {X@beta}")

GD(X, Y, beta0);

it:  0  mask: [ 1  1  1  1]  Z:[ 0  1 -3]  B: [ 0.0000 -0.1250  0.3750]  XB: [ 0.7500 -0.2500  0.5000  0.2500]
it:  1  mask: [ 1  1  1  1]  Z:[ 0  1 -3]  B: [ 0.0000 -0.1875  0.5625]  XB: [ 1.1250 -0.3750  0.7500  0.3750]


# 2. Pegasos

In [3]:
def pegasos(X, Y, beta0, lam=1, k=2, maxiter=2):
    beta = beta0.copy()
    t = 1
    for it in range(maxiter):
        nbatches = int(len(X)/k)
        x_batches = np.array_split(X, nbatches)
        y_batches = np.array_split(Y, nbatches)
        for x,y in zip(x_batches, y_batches):
            dbeta, mask, Z = grad(x,y,beta)
            beta -= (lam/t)*dbeta
            t += 1
            beta *= min(1, 1/(np.sqrt(lam)*norm(beta)) )
            print(F"it: {it:2d}  mask: {mask.astype(int)}  Z:{Z}  B: {beta}  XB: {X@beta}")

pegasos(X, Y, beta0, k=2)

it:  0  mask: [ 1  1]  Z:[ 0  2 -2]  B: [ 0.0000 -0.7071  0.7071]  XB: [ 1.4142 -1.4142  0.0000  0.0000]
it:  0  mask: [ 1  1]  Z:[ 0 -1 -1]  B: [ 0.0000 -0.1036  0.6036]  XB: [ 1.2071 -0.2071  1.0000  0.5000]
it:  1  mask: [ 0  1]  Z:[ 1  2  0]  B: [-0.1667 -0.4024  0.4024]  XB: [ 0.6381 -0.9714 -0.1667 -0.1667]
it:  1  mask: [ 1  1]  Z:[ 0 -1 -1]  B: [-0.1250 -0.1768  0.4268]  XB: [ 0.7286 -0.4786  0.3750  0.1250]


# 3. Dual Coordinate Descent

In [4]:
def dcd(X, Y, beta0, alpha0, lam=1, maxiter=2):
    beta = beta0.copy()
    alpha = alpha0.copy()
    Q = (X*Y[:,None]) @ (X*Y[:,None]).T
    D = np.diag(Q)
    C = 1/(len(X)*lam)
    for it in range(maxiter):
        for n,(a,x,y,q) in enumerate(zip(alpha, X, Y, D)):
            yhat = x @ beta
            dalpha = (y*yhat -1)/q
            alpha[n] = np.clip(a - dalpha, 0, C) 
            beta += (alpha[n] - a)*y*x
            print(F"it: {it:2d}  A: {alpha}  B: {beta}  XB: {X@beta}")
            
dcd(X, Y, beta0, alpha0)                       

it:  0  A: [ 0.2000  0.0000  0.0000  0.0000]  B: [ 0.2000  0.0000  0.4000]  XB: [ 1.0000  0.2000  1.0000  0.6000]
it:  0  A: [ 0.2000  0.2400  0.0000  0.0000]  B: [-0.0400 -0.4800  0.4000]  XB: [ 0.7600 -1.0000 -0.2000 -0.1200]
it:  0  A: [ 0.2000  0.2400  0.1333  0.0000]  B: [ 0.0933 -0.2133  0.6667]  XB: [ 1.4267 -0.3333  1.0000  0.5467]
it:  0  A: [ 0.2000  0.2400  0.1333  0.2500]  B: [-0.1567 -0.4633  0.4167]  XB: [ 0.6767 -1.0833 -0.2500 -0.2033]
it:  1  A: [ 0.2500  0.2400  0.1333  0.2500]  B: [-0.1067 -0.4633  0.5167]  XB: [ 0.9267 -1.0333  0.0000 -0.0533]
it:  1  A: [ 0.2500  0.2333  0.1333  0.2500]  B: [-0.1000 -0.4500  0.5167]  XB: [ 0.9333 -1.0000  0.0333 -0.0333]
it:  1  A: [ 0.2500  0.2333  0.2407  0.2500]  B: [ 0.0074 -0.2352  0.7315]  XB: [ 1.4704 -0.4630  1.0000  0.5037]
it:  1  A: [ 0.2500  0.2333  0.2407  0.2500]  B: [ 0.0074 -0.2352  0.7315]  XB: [ 1.4704 -0.4630  1.0000  0.5037]
